ref : https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/?utm_source=chatgpt.com

#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_Prophet.csv')
print(df.head())

#Preprocess

In [ ]:
df_prophet = df[['ds', 'y']].copy()

In [ ]:
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

In [ ]:
df_prophet.info()

In [ ]:
train_prophet = df_prophet.iloc[:-7]
test_prophet = df_prophet.iloc[-7:]

#Train

In [ ]:
# --- เทรนด้วยโหมด Default ---
model_add = Prophet(seasonality_mode='additive')
model_add.fit(train_prophet)

In [ ]:
future = model_add.make_future_dataframe(periods=7, freq='D')
# Filter out Saturdays (dayofweek == 5)
future_no_saturday = future[future['ds'].dt.dayofweek != 5].copy()
print("\nFuture dataframe created and Saturdays have been filtered out.")

In [ ]:
forecast = model_add.predict(future_no_saturday)
print("\nForecasting complete.")

#Evaluate

In [ ]:
# **ส่วนที่แก้ไข 1: กรองวันเสาร์ออกจากข้อมูลจริง (test_prophet) ด้วย**
actuals_no_saturday = test_prophet[test_prophet['ds'].dt.dayofweek != 5].copy()

# **ส่วนที่แก้ไข 2: เลือกผลการทำนายให้มีจำนวนเท่ากับข้อมูลจริงที่กรองแล้ว**
# forecast จะมีข้อมูลทั้งในอดีตและอนาคต เราจึงต้องเลือกมาเฉพาะส่วนท้าย
predictions = forecast['yhat'].tail(len(actuals_no_saturday))

print("-" * 50)
print(f"Number of actual points (after filtering): {len(actuals_no_saturday)}")
print(f"Number of predicted points: {len(predictions)}")
print("-" * 50)

In [ ]:
# **ส่วนที่แก้ไข 3: คำนวณ error จากข้อมูลที่กรองแล้วทั้งสองชุด**
if len(actuals_no_saturday) > 0:
    try:
        rmse = np.sqrt(mean_squared_error(actuals_no_saturday['y'], predictions))
        mae = mean_absolute_error(actuals_no_saturday['y'], predictions)

        print(f'RMSE (excluding Saturdays): {rmse:.3f}')
        print(f'MAE (excluding Saturdays): {mae:.3f}')

        # **ส่วนที่แก้ไข 4: พล็อตกราฟโดยใช้ข้อมูลที่กรองแล้วทั้งคู่**
        plt.figure(figsize=(12, 7))

        plt.plot(
            actuals_no_saturday['ds'],
            actuals_no_saturday['y'],
            label='Actual Sales (No Sat)',
            color='blue',
             marker='o',
            linestyle='-'
         )

        # forecast['ds'] ที่ไม่มีวันเสาร์ จะมี index ไม่ตรงกับ predictions (ซึ่งเป็นแค่ array ตัวเลข)
        # ดังนั้นเราจึงใช้ 'ds' จาก actuals_no_saturday เป็นแกน X แทนเพื่อให้ตรงกัน
        plt.plot(
            actuals_no_saturday['ds'],
            predictions.values,
            label='Prophet Forecast (No Sat)',
            color='red',
            marker='x',
            linestyle='-'
        )

        plt.title('Prophet Forecast vs Actual Sales (Excluding Saturdays)', fontsize=16)
        plt.xlabel('Date')
        plt.ylabel('Sales Sum')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    except FileNotFoundError:
        print(f"\n!!! ERROR: FILE NOT FOUND !!!")
        print(f"Please make sure you have uploaded '{file_path}' to your Colab session first.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("No data left to plot after filtering Saturdays.")